## Batch 2 - Coding Solution

# 1. Object Detection with YOLOv5 (Using Pretrained Weights)
# Problem: Detect objects using pre-trained YOLOv5 model.

# !pip install ultralytics
from ultralytics import YOLO

# Load model
model = YOLO('yolov5s.pt')  # Small YOLOv5 model

# Inference
results = model('sample.jpg')  # Image path

# Show predictions
results.show()


# 2.Preprocessing Image for ResNet Input
# Problem: Write a function to preprocess image for pretrained ResNet.

In [1]:
from torchvision import transforms
from PIL import Image

def preprocess_image(img_path):
    image = Image.open(img_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)


# 3. CNN for CIFAR-10 Classification
# Problem: Build a CNN model to classify CIFAR-10 images.

In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



2026-01-06 13:42:49.125544: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/harikrishnan/Documents/anaconda3/envs/myenv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# 4. Scenario: Face Mask Detection – Real-Time Webcam Classifier
# Problem: Build a real-time face mask detector using a trained CNN model and OpenCV.

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [6]:
# Dummy training data (100 images, 32x32 RGB)
x_train = np.random.rand(100, 32, 32, 3)
y_train = np.random.randint(0, 2, 100)

x_test = np.random.rand(20, 32, 32, 3)
y_test = np.random.randint(0, 2, 20)

# One-hot encode labels
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)


In [7]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])


/Users/harikrishnan/Documents/anaconda3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [11]:
model.fit(
    x_train,
    y_train,
    epochs=5,
    validation_data=(x_test, y_test)
)


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.5300 - loss: 0.7382 - val_accuracy: 0.3000 - val_loss: 0.9042
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5500 - loss: 0.7607 - val_accuracy: 0.3000 - val_loss: 0.7106
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5900 - loss: 0.6843 - val_accuracy: 0.7000 - val_loss: 0.6744
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5400 - loss: 0.6905 - val_accuracy: 0.3000 - val_loss: 0.7090
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5000 - loss: 0.7011 - val_accuracy: 0.3000 - val_loss: 0.7482


In [12]:
model.save("mask_detector.h5")
print("Model saved successfully")


Model saved successfully


In [13]:
loaded_model = load_model("mask_detector.h5")
print("Model loaded successfully")


Model loaded successfully


In [14]:
sample_image = np.random.rand(1, 32, 32, 3)
prediction = loaded_model.predict(sample_image)

predicted_class = np.argmax(prediction)
print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Predicted class: 1


# 5. Scenario: Custom Transfer Learning with Frozen + Trainable Layers
# Problem: Load a pretrained MobileNetV2 model, freeze base layers, add custom classifier, and fine-tune the top few layers.

In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in base_model.layers[:-20]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
